# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

Expected output:

>
>|    |   InvoiceNo |   StockCode |   year |   month |   day |   hour | Description                     |   Quantity | InvoiceDate         |   UnitPrice |   CustomerID | Country        |   amount_spent |
|---:|------------:|------------:|-------:|--------:|------:|-------:|:--------------------------------|-----------:|:--------------------|------------:|-------------:|:---------------|---------------:|
|  0 |      546084 |       22741 |   2011 |       3 |     3 |     11 | funky diva pen                  |         48 | 2011-03-09 11:28:00 |        0.85 |        14112 | United Kingdom |          40.8  |
|  1 |      545906 |       22557 |   2011 |       3 |     2 |      9 | plasters in tin vintage paisley |         12 | 2011-03-08 09:23:00 |        1.65 |        15764 | United Kingdom |          19.8  |
|  2 |      539475 |       22176 |   2010 |      12 |     7 |     14 | blue owl soft toy               |          1 | 2010-12-19 14:41:00 |        2.95 |        16686 | United Kingdom |           2.95 |
|  3 |      572562 |       21889 |   2011 |      10 |     2 |      9 | wooden box of dominoes          |         12 | 2011-10-25 09:07:00 |        1.25 |        13481 | United Kingdom |          15    |
|  4 |      549372 |       72741 |   2011 |       4 |     5 |     11 | grand chocolatecandle           |          9 | 2011-04-08 11:28:00 |        1.45 |        14958 | United Kingdom |          13.05 |

In [3]:
orders = pd.read_csv("../data/orders_sample.csv")

In [4]:
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.80
1,545906,22557,2011,3,2,9,plasters in tin vintage paisley,12,2011-03-08 09:23:00,1.65,15764,United Kingdom,19.80
2,539475,22176,2010,12,7,14,blue owl soft toy,1,2010-12-19 14:41:00,2.95,16686,United Kingdom,2.95
3,572562,21889,2011,10,2,9,wooden box of dominoes,12,2011-10-25 09:07:00,1.25,13481,United Kingdom,15.00
4,549372,72741,2011,4,5,11,grand chocolatecandle,9,2011-04-08 11:28:00,1.45,14958,United Kingdom,13.05


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [5]:
# Step 1: Aggregate the values by 'CustomerID' and return the sum
customers_df = orders.groupby('CustomerID').sum()

In [6]:
customers_df.head(1)

,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,
12347,3393172,12066,47,13,75,66,18.7,149.9


In [7]:
# select the columns 'amout_spend' and reset_index()
customers_df = customers_df['amount_spent'].reset_index()

In [8]:
customers_df.head(1)

,CustomerID,amount_spent
0,12347,149.9


In [9]:
# Step 2: amount_spent of to quantiles 75% and 95%
Q95 = customers_df['amount_spent'].quantile(0.95)
Q75 = customers_df['amount_spent'].quantile(0.75)

In [10]:
print(Q95)
print(Q75)

385.4600000000001
112.29000000000002


In [11]:
# Step 3: create a funtion to label the customers according to their amount_spent values  
def customer_group(row):
    if row['amount_spent'] >= Q95:
        return('VIP')
    elif row['amount_spent'] >= Q75 and row['amount_spent'] < Q95:
        return('Preferred')
    return('Regular')

In [17]:
# Step 4: Apply the function to the dataframe
customers_df['Group'] = customers_df.apply(lambda row: customer_group(row), axis=1)

In [13]:
# Other method

customers_df['Group'] = [customer_group(customers_df.iloc[i]) for i in range(len(customers_df))]

In [14]:
# Select the 'CustomerID' and the 'Group' variables
customers_group = customers_df[['CustomerID','Group']]

In [15]:
customers_group.head()

,CustomerID,Group
0,12347,Preferred
1,12348,Regular
2,12349,Regular
3,12350,Regular
4,12352,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [19]:
# SUB PROBLEM 1: Merge orders and customers_group_df 

orders_customer = pd.merge(
    # import left df
    orders, 
    # import right df
    customers_group, 
    # column's match
    on='CustomerID',
    # use keys from left frame only
    how='left')

In [20]:
orders_customer.head(1)

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Group
0,546084,22741,2011,3,3,11,funky diva pen,48,2011-03-09 11:28:00,0.85,14112,United Kingdom,40.8,Preferred


In [21]:
# Select the columns
columns = ['CustomerID','Country','Group']

In [23]:
# group the data by 'CustomerID','Country', and'Group'
orders_cust_group = orders_customer.groupby(columns).groups.keys()

# convert the dict_key element into a list
list_orders_cust_group = list(orders_cust_group)

# convert the list into a dataframe
country_cust = pd.DataFrame(list_orders_cust_group,
            columns=columns)

country_cust.head()

,CustomerID,Country,Group
0,12347,Iceland,Preferred
1,12348,Finland,Regular
2,12349,Italy,Regular
3,12350,Norway,Regular
4,12352,Norway,Preferred


In [24]:
# or

country_cust = orders_customer.loc[:, columns]
country_cust.head()

,CustomerID,Country,Group
0,14112,United Kingdom,Preferred
1,15764,United Kingdom,Preferred
2,16686,United Kingdom,Regular
3,13481,United Kingdom,Preferred
4,14958,United Kingdom,Regular


In [25]:
# For all customers, find those who are VIP.

condition = (orders_customer['Group'] == 'VIP')

country_vip = country_cust[condition]
country_vip.head()

,CustomerID,Country,Group
13,14646,Netherlands,VIP
26,12921,United Kingdom,VIP
30,17139,United Kingdom,VIP
34,12971,United Kingdom,VIP
38,12748,United Kingdom,VIP


In [26]:
# Count the number of customers that made orders from each country

country_vip = country_vip[['Country', 'Group']].groupby('Country').count()

country_vip.head()

,Group
Country,
Australia,54
Belgium,19
EIRE,375
France,124
Germany,94


In [27]:
# countries with the largest number of VIP customers

country_vip = country_vip.sort_values(by = 'Group', ascending = False)

country_vip.head()

,Group
Country,
United Kingdom,3562
EIRE,375
France,124
Netherlands,110
Germany,94


In [28]:
# Other method

In [29]:
country_cust.pivot_table(
    values='CustomerID',
    index='Country',
    columns='Group',
    aggfunc='count'

).sort_values(by = 'VIP', ascending = False).head()

Group,Preferred,Regular,VIP
Country,,,
United Kingdom,6532.0,7696.0,3562.0
EIRE,5.0,NaN,375.0
France,148.0,131.0,124.0
Netherlands,NaN,19.0,110.0
Germany,261.0,111.0,94.0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [30]:
# customers who are VIP or Preferre
country_vip_pref = country_cust[
    country_cust['Group'].isin(['VIP','Preferred'])]

In [31]:
country_vip_pref.head()

,CustomerID,Country,Group
0,14112,United Kingdom,Preferred
1,15764,United Kingdom,Preferred
3,13481,United Kingdom,Preferred
5,17602,United Kingdom,Preferred
8,13209,United Kingdom,Preferred


In [32]:
# number of customers by country
country_cust = country_vip_pref[[
    'Country', 'CustomerID']].groupby('Country').count()

In [33]:
# countries with the largest number of VIP and Preferred customers
country_cust.nlargest(1,'CustomerID')

,CustomerID
Country,
United Kingdom,10094


In [34]:
# countries with the largest number of VIP and Preferred customers
country_cust.sort_values(by = 'CustomerID', ascending = False).head()

,CustomerID
Country,
United Kingdom,10094
EIRE,380
Germany,355
France,272
Netherlands,110
